In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

TRAIN_DIR = 'mnist/input_data'
POS = 1
NEG = 7

In [2]:
# Import MNIST data
mnist = input_data.read_data_sets(TRAIN_DIR, one_hot=True)

Extracting mnist/input_data/train-images-idx3-ubyte.gz
Extracting mnist/input_data/train-labels-idx1-ubyte.gz
Extracting mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting mnist/input_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Get only two digits from MNIST
def get_binary_mnist(subset):
    X = []
    Y = []
    for _ in range(subset.num_examples):
        x, y = subset.next_batch(1)
        if y.flatten()[POS] == 1:
            X.append(x.flatten())
            Y.append([1])
        elif y.flatten()[NEG] == 1:
            X.append(x.flatten())
            Y.append([0])
    return np.asarray(X), np.asarray(Y)

X_train, Y_train = get_binary_mnist(mnist.train)
X_val, Y_val = get_binary_mnist(mnist.validation)
X_test, Y_test = get_binary_mnist(mnist.test)

In [4]:
# Create the model
x = tf.placeholder(tf.float32, [None, 28 * 28])
W = tf.Variable(tf.zeros([28 * 28, 1]))
b = tf.Variable(tf.zeros([1]))

# Logistic Regression Output (Sigmoid)
y = tf.divide(1, 1 + tf.exp(-tf.matmul(x, W) + b))

# Label
y_ = tf.placeholder(tf.float32, [None, 1])

# Binary Crossentropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1 - y)))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [7]:
# Training
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
batch_size = 100
for i in range(X_train.shape[0] // batch_size):
    batch_xs, batch_ys = X_train[i * batch_size:i * batch_size + batch_size], \
        Y_train[i * batch_size:i * batch_size + batch_size]
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# Test trained model
y_pred = sess.run(y, feed_dict={x: X_test, y_: Y_test})
roc_auc_score(Y_test, y_pred)

0.99609437940314371

In [6]:
# Create the model
n_hidden = 200
x = tf.placeholder(tf.float32, [None, 28 * 28])
W1 = tf.Variable(tf.zeros([28 * 28, n_hidden]))
b1 = tf.Variable(tf.zeros([n_hidden]))
W2 = tf.Variable(tf.zeros([n_hidden, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Logistic Regression Output (Sigmoid)
h = tf.divide(1, 1 + tf.exp(-tf.matmul(x, W1) + b1))
y = tf.divide(1, 1 + tf.exp(-tf.matmul(h, W2) + b2))

# Label
y_ = tf.placeholder(tf.float32, [None, 1])

# Binary Crossentropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1 - y)))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)